In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from matplotlib import pyplot

In [2]:
fnames = ["D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon1_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon2_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon3_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon4_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon5_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_1_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_2_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_3_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_4_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_5_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_6_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_7_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_8_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_9_Flow.csv",
          
         ]

In [3]:
dfList = list()
for fname in fnames: 
    data = pd.read_csv(fname)
    print('file:', fname, ' data size: ',data.shape)
    dfList.append(data)
############## combine the data frames
masterDF = pd.concat(dfList)
print(masterDF.shape)

file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon1_Flow.csv  data size:  (870, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon2_Flow.csv  data size:  (516, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon3_Flow.csv  data size:  (921, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon4_Flow.csv  data size:  (467, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon5_Flow.csv  data size:  (800, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_1_Flow.csv  data size:  (1420, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_2_Flow.csv  data size:  (52, 84)
file: D:/New pcaps_IoTSpyCamer

In [4]:
#see unique labels
masterDF['Label'].unique()
masterDF['Label'].replace('VConf_Gmeet','VConf_GMeet',inplace=True)
uniqueLabels = list(masterDF['Label'].unique())
print(uniqueLabels)

['VShare_Amazon', 'VShare_YouTube']


In [5]:
#discard all those columns that helps in uniquely identifying a flow
masterPruned = masterDF.iloc[:,np.arange(7,84)]
print('pruneddata size: ', masterPruned.shape)
ftList = list(masterPruned.columns)
print(ftList)

pruneddata size:  (7880, 77)
['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg

In [6]:
#discard those flows that has 0 flow duration
masterSelected = masterPruned.drop(masterPruned[masterPruned['Flow Duration']==0].index)
masterSelected.shape

(5285, 77)

In [7]:
#identify those columns that has non-numeric values
typesList = masterSelected.dtypes
ftCount = len(ftList)
nonNumericFtList = list()
for fi in range(ftCount):
    if ftList[fi] != 'Label' and typesList[fi] != 'int64' and typesList[fi] != 'float64':
        print(ftList[fi], '  ', typesList[fi])
        nonNumericFtList.append(ftList[fi])
print(nonNumericFtList)

[]


In [8]:
#pyplot.boxplot(results, labels=names, showmeans=True)
#pyplot.scatter(fpackets,fbytes)
#pyplot.show()


In [9]:
for ft in nonNumericFtList:
    masterSelected[ft] = pd.to_numeric(masterSelected[ft], errors='coerce')

In [10]:
print(nonNumericFtList)
for ft in nonNumericFtList:
    countNaN = masterSelected[ft].isna().sum()
    print('converted to numeric ft: ', ft, '  countNaN: ', countNaN)

[]


In [11]:
number_inf = masterSelected[masterSelected == np.inf].count().sum()
print('count inf: ',number_inf)

count inf:  0


In [12]:
typesList = masterSelected.dtypes
ftCount = len(ftList)
nonNumericFtList = list()
for fi in range(ftCount):
    if ftList[fi] != 'Label' and typesList[fi] != 'int64' and typesList[fi] != 'float64':
        print(ftList[fi], '  ', typesList[fi])
        nonNumericFtList.append(ftList[fi])
print(nonNumericFtList)

[]


In [13]:
#identify those columns that has 0 std
count = 0
staticFtList = list()
for ft in ftList:
    if ft == 'Label':
        continue
    #print('ft : ', ft)
    ftMin = masterSelected[ft].min()
    ftMax = masterSelected[ft].max()
    #std = masterPruned[ft].std()
    
    if ftMin == ftMax:
        count += 1
        print('count: ', count, ' ft: ', ft, ' ftmin: ', ftMin, ' ftMax: ', ftMax)
        staticFtList.append(ft)
print(staticFtList)

count:  1  ft:  Fwd PSH Flags  ftmin:  0  ftMax:  0
count:  2  ft:  Fwd URG Flags  ftmin:  0  ftMax:  0
count:  3  ft:  Bwd URG Flags  ftmin:  0  ftMax:  0
count:  4  ft:  URG Flag Cnt  ftmin:  0  ftMax:  0
count:  5  ft:  CWE Flag Count  ftmin:  0  ftMax:  0
count:  6  ft:  ECE Flag Cnt  ftmin:  0  ftMax:  0
count:  7  ft:  Fwd Byts/b Avg  ftmin:  0  ftMax:  0
count:  8  ft:  Fwd Pkts/b Avg  ftmin:  0  ftMax:  0
count:  9  ft:  Fwd Blk Rate Avg  ftmin:  0  ftMax:  0
count:  10  ft:  Bwd Byts/b Avg  ftmin:  0  ftMax:  0
count:  11  ft:  Bwd Pkts/b Avg  ftmin:  0  ftMax:  0
count:  12  ft:  Bwd Blk Rate Avg  ftmin:  0  ftMax:  0
count:  13  ft:  Init Fwd Win Byts  ftmin:  -1  ftMax:  -1
count:  14  ft:  Fwd Seg Size Min  ftmin:  0  ftMax:  0
['Fwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Init Fwd Win Byts', 'Fwd Seg Size 

In [14]:
#remove those columns that have 0 std
print('before droping column: ', masterSelected.shape)
for ft in staticFtList:
    masterSelected.drop([ft], axis=1,inplace=True)
print('after droping column: ', masterSelected.shape)    


before droping column:  (5285, 77)
after droping column:  (5285, 63)


In [15]:
xCount = masterSelected.shape[1]
print(xCount)
finalFtList = list(masterSelected.columns)
print(finalFtList)

63
['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Bwd PSH Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle 

In [16]:
#report intermediate data frame to a csv
#masterSelected.to_csv('../reports_db/CICFlowMeter_VideoConferencingOutput/ProcessedNPrunnedAllConf.csv', index = False)

In [17]:
#classification with all the features
masterDS = masterSelected.values
X = masterDS[:,:-1]
Y = masterDS[:,-1]
print("Xshape: ", X.shape)
print("Yshape: ", Y.shape)

Xshape:  (5285, 62)
Yshape:  (5285,)


In [18]:
# feature selection using ANNOVA F-score
topKval = 10
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=topKval)
# fit RFE
rfe.fit(X, Y)
# summarize all features

# transform train input data
XNew = rfe.transform(X)
# transform test input data
print('XNew.shape', XNew.shape)
#print('XNew', XNew)

XNew.shape (5285, 10)


In [19]:
ftSelected = list()#features[0:-1]
for fi in range(xCount-1):
    ft  = finalFtList[fi]
    
    if ft != 'Label' and rfe.support_[fi]:
        #print('i = ', fi, ' ftName: ', ftList[fi], ' FS.support_: ', FS.get_support()[fi], ' FS.scores: = ', FS.scores_[fi])
        #scores.append(ref.scores_[fi])
        ftSelected.append(ft)
        print('Column: %d, ColName: %s, Selected %s, Rank: %.3f' % (fi, finalFtList[fi], rfe.support_[fi], rfe.ranking_[fi]))
print(ftSelected)

Column: 0, ColName: Flow Duration, Selected True, Rank: 1.000
Column: 5, ColName: Fwd Pkt Len Max, Selected True, Rank: 1.000
Column: 15, ColName: Flow IAT Mean, Selected True, Rank: 1.000
Column: 16, ColName: Flow IAT Std, Selected True, Rank: 1.000
Column: 17, ColName: Flow IAT Max, Selected True, Rank: 1.000
Column: 23, ColName: Fwd IAT Min, Selected True, Rank: 1.000
Column: 25, ColName: Bwd IAT Mean, Selected True, Rank: 1.000
Column: 27, ColName: Bwd IAT Max, Selected True, Rank: 1.000
Column: 28, ColName: Bwd IAT Min, Selected True, Rank: 1.000
Column: 52, ColName: Init Bwd Win Byts, Selected True, Rank: 1.000
['Flow Duration', 'Fwd Pkt Len Max', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Max', 'Bwd IAT Min', 'Init Bwd Win Byts']


In [20]:
clf = DecisionTreeClassifier(random_state=0)
print('classification with top 10 selected features:')
scores = cross_val_score(clf, XNew, Y, cv=10)
print(scores)
print('mean score: ', scores.mean(),'min score: ', scores.min(), 'max score: ', scores.max(), ' std score: ', scores.std())

classification with top 10 selected features:
[0.93005671 0.96597353 0.97164461 0.89603025 0.9168242  0.72916667
 0.81628788 0.83333333 0.78598485 0.8219697 ]
mean score:  0.8667271724809533 min score:  0.7291666666666666 max score:  0.9716446124763705  std score:  0.0770332656266717


In [21]:
#performance with to k  features using decision tree
X_train, X_test, Y_train, Y_test = train_test_split(XNew, Y, test_size=0.33, random_state=1)
model = DecisionTreeClassifier(random_state=0)
clf = model.fit(X_train,Y_train)
ypred = model.predict(X_test)

In [22]:
from sklearn.tree import export_text
from sklearn import tree

text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_7 <= 23.50
|   |--- feature_9 <= 507.50
|   |   |--- feature_5 <= 306825.00
|   |   |   |--- feature_1 <= 1355.00
|   |   |   |   |--- feature_5 <= 312.50
|   |   |   |   |   |--- class: VShare_YouTube
|   |   |   |   |--- feature_5 >  312.50
|   |   |   |   |   |--- feature_4 <= 376.00
|   |   |   |   |   |   |--- class: VShare_Amazon
|   |   |   |   |   |--- feature_4 >  376.00
|   |   |   |   |   |   |--- feature_9 <= 56.50
|   |   |   |   |   |   |   |--- feature_5 <= 218322.00
|   |   |   |   |   |   |   |   |--- class: VShare_Amazon
|   |   |   |   |   |   |   |--- feature_5 >  218322.00
|   |   |   |   |   |   |   |   |--- class: VShare_YouTube
|   |   |   |   |   |   |--- feature_9 >  56.50
|   |   |   |   |   |   |   |--- class: VShare_YouTube
|   |   |   |--- feature_1 >  1355.00
|   |   |   |   |--- class: VShare_Amazon
|   |   |--- feature_5 >  306825.00
|   |   |   |--- feature_4 <= 2011913.00
|   |   |   |   |--- feature_3 <= 521870.78
|   |   |   |   |   |--

In [23]:
print(clf.tree_.max_depth)

22


In [90]:
share_mat = metrics.confusion_matrix(Y_test, ypred,labels=uniqueLabels)
#[i][j] ==> knwon to be in group i and predicted as group j
print(share_mat)
print('Amazon as Amazon = ',share_mat[0][0],' percent = ', (share_mat[0][0]/(share_mat[0][0]+share_mat[0][1])))
print('Amazon as Youtube = ',share_mat[0][1],' percent = ', (share_mat[0][1]/(share_mat[0][0]+share_mat[0][1])))
print('\n')

print('Youtube as Amazon = ',share_mat[1][0],' percent = ', (share_mat[1][0]/(share_mat[1][0]+share_mat[1][1])))
print('Youtube as Youtube = ',share_mat[1][1],' percent = ', (share_mat[1][1]/(share_mat[1][0]+share_mat[1][1])))



[[652  91]
 [107 895]]
Amazon as Amazon =  652  percent =  0.8775235531628532
Amazon as Youtube =  91  percent =  0.1224764468371467


Youtube as Amazon =  107  percent =  0.10678642714570859
Youtube as Youtube =  895  percent =  0.8932135728542914


In [91]:
#classification with unselected and uncorrelated features
corr = masterSelected.corr()
corr.shape

(62, 62)

In [92]:
corrThreshold = 0.65
unCorrUnSelFtList = list()
for ft1 in finalFtList:#baseft = 'Idle Max' for each of the selected features
    #print('ft1: ', ft1)
    isCorr = False
    for ft2 in ftSelected:# for each of the top 10 features
        if ft1 == 'Label' or ft2 == 'Label' or (ft1 == ft2):
            continue
        elif abs(corr[ft1][ft2]) > corrThreshold:
                isCorr = True
                print('ft1: ', ft1, 'ft2: ', ft2, '  corr: ', corr[ft1][ft2])
                break
    if not isCorr and ft1 not in unCorrUnSelFtList:
        #print('corr[{0}][{1}] = {2}: isPresentInRestFtlist: {3}'.format(baseft,ft,corr[baseft][ft], ft in restFtList))
        unCorrUnSelFtList.append(ft1)
        
    #elif abs(corr[ft1][ft] >= 0.6 and ft in unCorrUnSelFtList):
    #            unCorrUnSelFtList.remove(ft)
print(len(unCorrUnSelFtList))
print(unCorrUnSelFtList)

print(len(ftSelected))
print(ftSelected)

ft1:  Flow Duration ft2:  Flow IAT Std   corr:  0.6785223885123135
ft1:  Fwd Pkt Len Std ft2:  Fwd Pkt Len Max   corr:  0.848435839051745
ft1:  Flow IAT Std ft2:  Flow Duration   corr:  0.6785223885123135
ft1:  Flow IAT Max ft2:  Flow Duration   corr:  0.7895824858160128
ft1:  Flow IAT Min ft2:  Flow IAT Mean   corr:  0.8259749428461993
ft1:  Fwd IAT Tot ft2:  Flow Duration   corr:  0.9363974518404051
ft1:  Fwd IAT Mean ft2:  Flow IAT Std   corr:  0.7968598350927399
ft1:  Fwd IAT Std ft2:  Flow IAT Std   corr:  0.6747473090596495
ft1:  Fwd IAT Max ft2:  Flow Duration   corr:  0.775965750693583
ft1:  Bwd IAT Tot ft2:  Flow Duration   corr:  0.9282755995528241
ft1:  Bwd IAT Mean ft2:  Flow IAT Std   corr:  0.8312587612581815
ft1:  Bwd IAT Std ft2:  Flow Duration   corr:  0.6652218093626003
ft1:  Bwd IAT Max ft2:  Flow Duration   corr:  0.7547757306955687
ft1:  Idle Mean ft2:  Flow Duration   corr:  0.7462183863174373
ft1:  Idle Max ft2:  Flow Duration   corr:  0.7814280112992654
ft1:  Id

In [93]:
for ft1 in ftSelected:
    for ft2 in unCorrUnSelFtList:
        if ft1 != 'Label' or ft2 == 'Label':
            continue
        if abs(corr[ft][ft2]) >= corrThreshold:
            print('ft: ',ft, ' ft2: ',ft2, corr[ft][ft2])

In [94]:
#slice master data frame to only uncorrelated and unselected features
masterUnSelected = masterSelected[unCorrUnSelFtList]
print(masterUnSelected.shape)

(5285, 47)


In [95]:
#### classification with unselected columns
masterUnSelDS = masterUnSelected.values
Xnotsel = masterUnSelDS[:,:-1]
Y = masterUnSelDS[:,-1]
print("Xshape: ", Xnotsel.shape)
print("Yshape: ", Y.shape)

Xshape:  (5285, 46)
Yshape:  (5285,)


In [96]:
#find cross validation score with uncorrelated and unselecte feature data frame
clf = DecisionTreeClassifier(random_state=0)
print('classification with rest (not top 10) features:')
scores = cross_val_score(clf, Xnotsel, Y, cv=10)
print(scores)
print('mean score: ', scores.mean(), ' std score: ', scores.std())

classification with rest (not top 10) features:
[0.93383743 0.96030246 0.96786389 0.93761815 0.92438563 0.84469697
 0.83901515 0.85984848 0.74810606 0.83522727]
mean score:  0.8850901500830612  std score:  0.06695566450462052


In [97]:
#classification with with uncorrelated and unselecte feature data frame
X_train, X_test, Y_train, Y_test = train_test_split(Xnotsel, Y, test_size=0.33, random_state=1)
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train,Y_train)
yhat = model.predict(X_test)

In [98]:
share_mat = metrics.confusion_matrix(Y_test, yhat,labels=uniqueLabels)
#[i][j] ==> knwon to be in group i and predicted as group j
print(share_mat)
print('Amazon as Amazon = ',share_mat[0][0],' percent = ', (share_mat[0][0]/(share_mat[0][0]+share_mat[0][1])))
print('Amazon as Youtube = ',share_mat[0][1],' percent = ', (share_mat[0][1]/(share_mat[0][0]+share_mat[0][1])))
print('\n')

print('Youtube as Amazon = ',share_mat[1][0],' percent = ', (share_mat[1][0]/(share_mat[1][0]+share_mat[1][1])))
print('Youtube as Youtube = ',share_mat[1][1],' percent = ', (share_mat[1][1]/(share_mat[1][0]+share_mat[1][1])))



[[654  89]
 [100 902]]
Amazon as Amazon =  654  percent =  0.8802153432032301
Amazon as Youtube =  89  percent =  0.11978465679676985


Youtube as Amazon =  100  percent =  0.0998003992015968
Youtube as Youtube =  902  percent =  0.9001996007984032
